In [1]:
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.core import StorageContext, load_index_from_storage
from google.oauth2 import service_account
from llama_index.llms.vertex import Vertex
from llama_index.embeddings.vertex import VertexTextEmbedding
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex
import os

username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
url = os.getenv("NEO4J_URI")

In [3]:
documents = SimpleDirectoryReader(
     input_files=["data/Graph_Retrieval-Augmented_Generation_A_Survey.pdf"]
).load_data()

In [4]:
filename = "caselaws-421916-cc2ca960f520.json"
credentials: service_account.Credentials = (
    service_account.Credentials.from_service_account_file(filename)
)

In [5]:
llm = Vertex(
        model="gemini-pro", 
        project=credentials.project_id, credentials=credentials,
        max_tokens=8000
    )
    
Settings.llm = llm
Settings.chunk_size = 512

In [6]:
embed_model = VertexTextEmbedding(
    model_name="textembedding-gecko@003",
    project=credentials.project_id, credentials=credentials
)

Settings.embed_model = embed_model

In [7]:
# Extract graph from documents
embed_dim = 768

neo4j_vector_hybrid = Neo4jVectorStore(
    username, password, url, embed_dim, hybrid_search=True
)

storage_context = StorageContext.from_defaults(
    vector_store=neo4j_vector_hybrid
)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setVectorProperty(c, 'embedding', row.embedding) YIELD node SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: lin

In [8]:
index.storage_context.persist(persist_dir="./storage_hybrid_gemini")
index = load_index_from_storage(storage_context)

In [9]:
# Define retriever
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)
results = retriever.retrieve("What is Graph Retrieval-Augmented Generation?")
for record in results:
    print(record.text)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

Graph-Based Indexing (G-Indexing). Graph-Based Indexing constitutes the initial phase of
GraphRAG, aimed at identifying or constructing a graph database Gthat aligns with downstream
tasks and establishing indices on it. The graph database can originate from public knowledge
graphs [ 2,7,91,131,138,151], graph data [ 112], or be constructed based on proprietary data
sources such as textual [ 25,43,80,160] or other forms of data [ 169]. The indexing process typi-
cally includes mapping node and edge properties, establishing pointers between connected nodes,
and organizing data to support fast traversal and retrieval operations. Indexing determines the
granularity of the subsequent retrieval stage, playing a crucial role in enhancing query efficiency.
Graph-Guided Retrieval (G-Retrieval). Following graph-based indexing, the graph-guided retrieval
phase focuses on extracting pertinent information from the graph database in response to user
queries or input. Specifically, given a user query

In [10]:
# Question answering
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query("What is Graph Retrieval-Augmented Generation?")
display(Markdown(f"{response}"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

## Graph Retrieval-Augmented Generation (GraphRAG)

Graph Retrieval-Augmented Generation (GraphRAG) is a framework that combines graph-based indexing, retrieval, and generation to process information and generate responses. It consists of three main phases:

**1. Graph-Based Indexing (G-Indexing):**

* This phase focuses on identifying or constructing a graph database (G) that aligns with the downstream tasks.
* The graph database can be sourced from public knowledge graphs, graph data, or constructed from proprietary data sources like textual or other forms of data.
* The indexing process involves mapping node and edge properties, establishing pointers between connected nodes, and organizing data for efficient traversal and retrieval.
* This phase determines the granularity of the subsequent retrieval stage and plays a crucial role in enhancing query efficiency.

**2. Graph-Guided Retrieval (G-Retrieval):**

* This phase focuses on extracting relevant information from the graph database in response to user queries or input.
* Given a user query (q) expressed in natural language, the retrieval stage aims to extract the most relevant elements (e.g., entities, triplets, paths, subgraphs) from the knowledge graph.
* This can be formulated as finding the optimal retrieved graph elements (G*) that maximize the semantic similarity between the user query and the graph data.
* The search range of subgraphs is narrowed down using a function R(·) to ensure efficiency.

**3. Graph-Enhanced Generation (G-Generation):**

* This phase involves synthesizing meaningful outputs or responses based on the retrieved graph data.
* This could encompass answering user queries, generating reports, etc.

Overall, GraphRAG leverages the power of graph databases to enhance information retrieval and generation tasks, leading to more accurate and efficient results.

In [11]:
response = query_engine.query("How to do Query-Focused Summarization (QFS)?")
display(Markdown(f"{response}"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

## Query-Focused Summarization (QFS)

While the provided context doesn't directly address QFS techniques, it does mention Graph Retrieval-Augmented Generation (GraphRAG) as a solution for tasks like QFS. 

Here's what we can glean about QFS from the context:

**Challenges:**

* Traditional RAG models struggle with QFS due to limitations in retrieving relevant information.
* They often rely on a subset of documents and fail to capture global information comprehensively.

**GraphRAG as a solution:**

* GraphRAG retrieves graph elements containing relational knowledge pertinent to a given query from a pre-constructed graph database.
* This allows for a more accurate and comprehensive retrieval of information, including interconnections between texts.
* Graph data offers abstraction and summarization of textual data, mitigating concerns of verbosity.
* By retrieving subgraphs or graph communities, GraphRAG can access comprehensive information to effectively address the QFS challenge.

**Further research:**

The context highlights the need for further research in GraphRAG, particularly in areas like:

* **Graph-Based Indexing (G-Indexing):** Optimizing methods for indexing and storing graph data for efficient retrieval.
* **Graph-Guided Retrieval (G-Retrieval):** Developing advanced techniques for retrieving relevant graph elements based on the query and context.
* **Graph-Enhanced Generation (G-Generation):** Utilizing retrieved graph information to generate summaries that are both informative and focused on the query.

**Additional resources:**

While the context doesn't provide specific QFS techniques, it offers valuable insights into the potential of GraphRAG for this task. 

For further information on QFS, you can explore the following resources:

* **A Survey on Complex Knowledge Base Question Answering: Methods, Challenges and Solutions:** This paper provides a comprehensive overview of QFS methods, including those based on graph neural networks.
* **Query Graph Generation for Answering Multi-hop Complex Questions from Knowledge Bases:** This paper explores the use of query graphs for QFS in the context of knowledge base question answering.
* **Natural Questions: a Benchmark for Question Answering Research:** This paper introduces the Natural Questions dataset, a large-scale benchmark for QFS and other question answering tasks.

By combining the insights from the provided context with these additional resources, you can gain a deeper understanding of QFS and explore the potential of GraphRAG for this task.